In [17]:
import mdtraj as md

molecule="gtt"
molecule_lower = molecule.lower()
molecule_upper = molecule.upper()

In [19]:
traj = md.load(f"../data/{molecule_upper}/folded.pdb")
traj.save_gro(f"../data/{molecule_upper}/folded.gro")

In [ ]:
print("\nCreate .gro file")
# print(f"gmx pdb2gmx -f {molecule_upper}/folded.pdb -o {molecule_upper}/folded.gro -p {molecule_upper}/topol.top -ignh")
print(f"gmx pdb2gmx -f {molecule_upper}/folded.pdb -o {molecule_upper}/folded.gro -p {molecule_upper}/topol.top -ff charmm27 -water tip3p --ignh")

print("\nCreate .tpr file")
print(f"gmx grompp -f {molecule_lower}.mdp -c {molecule_upper}/folded.gro -p {molecule_upper}/topol.top -o {molecule_upper}/nvt_0.tpr -maxwarn 2")


Create .gro file
gmx pdb2gmx -f GTT/folded.pdb -o GTT/folded.gro -p GTT/topol.top -ff charmm27 -water tip3p --ignh

Create .tpr file
gmx grompp -f gtt.mdp -c GTT/folded.gro -p GTT/topol.top -o GTT/gtt.tpr -maxwarn 2


In [ ]:
gmx grompp -f test.mdp -c 2JOF/folded.gro -p 2JOF/topol.top -o 2jof.tpr

gmx pdb2gmx -f 2JOF/folded.pdb -o 2JOF/folded.gro -p 2JOF/topol.top -ff amber99sb-ildn -water tip3p

gmx grompp -f 2JOF/2jof.mdp -c 2JOF/folded.gro -p 2JOF/topol.top -o 2jof.tpr

gmx grompp -f test.mdp -c 2JOF/folded.gro -p 2JOF/topol.top -o 2jof.tpr

In [ ]:
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/atomtypes.atp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/lipids.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.rtp
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/lipids.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.hdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.n.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.c.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/dna.c.tdb
Opening force field file /gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/rna.c.tdb

/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/atomtypes.atp
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.hdb
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.rtp
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.n.tdb
/gromacs/AVX2_256_ts/share/gromacs/top/charmm27.ff/aminoacids.c.tdb

residuetypes.dat 

In [ ]:
1.79071   2.10721   1.56292

8: CHARMM27 all-atom force field (CHARM22 plus CMAP for proteins)

1: TIP3P   TIP 3-point, recommended